In [111]:
library(tidyverse)
library(tidyr)
library(MASS)

In [117]:
set.seed(20230522)

n_samples <- 1000
n_units <- 20
G <- 10
beta_1 <- 5
beta_2 <- 8

m <- 0
sigma <- 2

pop_i <- sample(1000:1000000, size = n_units)
pop_j <- sample(1000:1000000, size = n_units)
pos_i <- runif(n_units, min = 0, max = 1000)
pos_j <- runif(n_units, min = 0, max = 1000)

# Generate data
data <- data.frame(i = sample(1:n_units, size = n_samples, replace = TRUE),
                   j = sample(1:n_units, size = n_samples, replace = TRUE))

# Generates new i and j values when they are the same for each row
data <- data %>%
    mutate(i = ifelse(i == j, sample(1:n_units, size = n_samples, replace = TRUE), i),
           j = ifelse(i == j, sample(1:n_units, size = n_samples, replace = TRUE), j))

data$pop_i <- pop_i[data$i]
data$pop_j <- pop_i[data$j]
data$distance <- abs(pos_i[data$i] - pos_j[data$j])

data$Fij <- (G * data$pop_i^beta_1 * data$pop_j^beta_2 / data$distance^sigma)
data$Fij <- data$Fij * exp(rnorm(n_samples, mean = m, sd = 0.01))

head(data)

,i,j,pop_i,pop_j,distance,Fij
,<int>,<int>,<int>,<int>,<dbl>,<dbl>
1,20,12,201382,761887,155.0594,1.551119e+70
2,14,2,409122,255275,121.2057,1.418954e+68
3,2,4,255275,626873,257.0906,3.930197e+69
4,9,7,162696,934961,382.3464,4.501573e+69
5,4,11,626873,760746,366.5726,7.928176e+71
6,14,2,409122,255275,121.2057,1.416924e+68


In [118]:
model <- glm(Fij ~ log(pop_i) + log(pop_j) + log(distance), data = data, family = gaussian(link = log))
summary(model)


Call:
glm(formula = Fij ~ log(pop_i) + log(pop_j) + log(distance), 
    family = gaussian(link = log), data = data)

Deviance Residuals: 
       Min          1Q      Median          3Q         Max  
-7.778e+72  -1.896e+67  -2.033e+64  -4.014e+59   4.457e+72  

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)    2.475921   0.204910   12.08   <2e-16 ***
log(pop_i)     4.996586   0.006869  727.39   <2e-16 ***
log(pop_j)     7.990613   0.009370  852.76   <2e-16 ***
log(distance) -1.999680   0.002163 -924.36   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for gaussian family taken to be 1.3139e+143)

    Null deviance: 1.6043e+150  on 999  degrees of freedom
Residual deviance: 1.3086e+146  on 996  degrees of freedom
AIC: 332387

Number of Fisher Scoring iterations: 3


In [119]:
c1 <- model$coefficients[[1]]
c2 <- model$coefficients[[2]]
c3 <- model$coefficients[[3]]
c4 <- model$coefficients[[4]]

coefficients_df <- data.frame(
  Variable = c("c1", "c2", "c3", "c4"),
  Found_Values = c(exp(c1), c2, c3, c4),
  Real_Values = c(G, beta_1, beta_2, -sigma)
)

coefficients_df 

Variable,Found_Values,Real_Values
<chr>,<dbl>,<dbl>
c1,11.892659,10
c2,4.996586,5
c3,7.990613,8
c4,-1.999680,-2
